the classes will eventually be split up but they are put together for now to get their interactions correct <br>
sources: <br>
events: https://www.quantstart.com/articles/Event-Driven-Backtesting-with-Python-Part-II/ <br>
portfolio: https://www.quantstart.com/articles/Event-Driven-Backtesting-with-Python-Part-V/ <br>
execution handler: https://www.quantstart.com/articles/Event-Driven-Backtesting-with-Python-Part-VI/

In [ ]:
import numpy as np
import pandas as pd
import os

from abc import ABC, abstractmethod
from queue import PriorityQueue
from dataclasses import dataclass

In [ ]:
quotes_filename = os.path.join("play_data", "XBTUSD_quotes_191214_0434.csv")
trades_filename = os.path.join("play_data", "XBTUSD_trades_191214_0434.csv")

In [ ]:
quotes_full = pd.read_csv(
    quotes_filename,
    index_col='recorded',
    parse_dates=['timestamp', 'recorded']
)

trades_full = pd.read_csv(
    trades_filename,
    index_col='received',
    parse_dates=['timestamp', 'received'],
    dtype={"side":"category"}
)

all_data = {
    "quotes": quotes_full,
    "trades": trades_full
}

display( quotes_full.head() )
display( trades_full.head() )

In [ ]:
@dataclass
class FillEvent:
    __slots__ = "event_time", "symbol", "exchange", "quantity", "cost"
    event_time : pd.Timestamp
    symbol: str
    exchange: str
    quantity: int  # change in securities held
    cost: float  # change in cash held
        
    @property
    def commission(self):
        # will eventually contain exchange/broker-specific logic
        return self.cost * 0.005
    
@dataclass
class Signal:
    __slots__ = "symbol", "exchange", "signal_type", "strength"
    symbol: str
    exchange: str
    signal_type: str  # BUY/SELL/EXIT
    strength: float
    
@dataclass
class Order:
    __slots__ = "symbol", "exchange", "order_type", "direction", "quantity", "price"
    symbol: str
    exchange: str
    order_type: str  # MKT/LMT
    direction: str  # BUY/SELL
    quantity: int
    price: float  # only relevent to LMT

In [ ]:
class Portfolio(ABC):
    @abstractmethod
    def update_fill(self):
        pass
    
    @abstractmethod
    def send_order_from_signal(self):
        pass
    
class BacktestPortfolio(Portfolio):
    def __init__(self, queue, start_time, symbol_list, all_data, initial_capital=10000.0):
        self.queue = queue  # event queue
        self.symbol_list = symbol_list 
        self.initial_capital = initial_capital
        self.current_time = start_time
        
        self.all_positions_list = [pd.Series(0, index=symbol_list, name=start_time)]  # list of series
        self.all_holdings_list = [self.construct_current_holdings(start_time)]  # list of series
        
        self.current_positions = pd.Series(0, index=symbol_list, name=start_time)
        self.current_holdings = self.construct_current_holdings(start_time)  # series

    @property
    def all_positions(self):
        return pd.DataFrame(self.all_positions_list)
    
    @property
    def all_holdings(self):
        return pd.DataFrame(self.all_holdings_list)
    
    def construct_current_holdings(self, time):  # market values of positions
        holdings = {symbol: 0.0 for symbol in symbol_list}
        holdings["capital"] = self.initial_capital
        holdings["commission"] = 0.0
        holdings["total"] = self.initial_capital
        return pd.Series(holdings, name=time)
    
    def update_fill(self, fill):
        self.current_time = fill.event_time
        
        # maybe can split position and holding updates
        self.current_positions.name = self.current_time
        self.current_positions[fill.symbol] += fill.quantity
        self.all_positions_list.append(self.current_positions.copy())
        
        self.current_holdings.name = self.current_time
        self.current_holdings["capital"] -= fill.cost
        # assuming constant prices between txns
        self.current_holdings[fill.symbol] = self.current_positions[fill.symbol] * fill.cost
        self.current_holdings["commission"] += fill.commission
        self.all_holdings_list.append(self.current_holdings.copy())
        
    def risk_check(self):
        return True
    
    def calculate_order_price(self, order_type):
        # relevant to limit orders
        if order_type == "MKT":
            return 0.0
        return 0.0
    
    def send_order_from_signal(self, sig):
        print(sig)
        
        order_type = "MKT"
        
        if sig.signal_type == "EXIT":
            if self.current_positions[signal.symbol] < 0:
                direction = "BUY"
            else:
                direction = "SELL"
        else:
            direction = sig.signal_type
            
        quantity = int(10 * sig.strength)
        price = self.calculate_order_price(order_type)
        
        if self.risk_check():
            self.generate_fill_from_order(
                Order(sig.symbol, sig.exchange, order_type, direction, quantity, price)
            )
    
    def calculate_fill_cost(self, order):
        fill_price = 7000  # will eventually use future data
        return order.quantity * fill_price

    def calculate_fill_quantity(self, order):
        return order.quantity  # assumes 100% fill

    def generate_fill_from_order(self, order):
        print(order)
        event_time = self.current_time + pd.Timedelta("3s")
        quantity = self.calculate_fill_quantity(order)
        cost = self.calculate_fill_cost(order)
        
        fill = FillEvent(event_time, order.symbol, order.exchange, quantity, cost)
        print(fill)
        self.queue.put((event_time, fill))
        event_time += pd.Timedelta("1s")
        fill = FillEvent(event_time, order.symbol, order.exchange, quantity, cost)
        print(fill)
        self.queue.put((event_time, fill))
    
symbol_list = "XBTUSD", "ETHUSD"
time0 = pd.Timestamp.utcnow() - pd.Timedelta("15s")
eq = PriorityQueue()

p = BacktestPortfolio(eq, time0, symbol_list, all_data, 1000000)
s = Signal("XBTUSD", "BitMex", "BUY", 5.0)

p.send_order_from_signal(s)

while not eq.empty():
    event_time, event = eq.get()
    if type(event) == FillEvent:
        p.update_fill(event)
    display(p.all_positions)
    display(p.all_holdings)